# LLM (Large Language Model) yang mampu membaca dan menganalisis isi konten dari berbagai format file

Dalam kasus ini bertujuan membangun sistem berbasis llm yaitu
1. Memahami konten teks dari file
2. menjawab berdasarkan konten file dengan pemanfaatan RAG

In [2]:
!pip install langchain langchain-community faiss-cpu pdfplumber sentence-transformers tqdm langchainhub chromadb langchain-ollama -q


## ekstraksi teks file 

In [44]:
import pdfplumber

def extract_text_doc(file):
    with pdfplumber.open(file) as pdf:
        return "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])

pdf_text_ext = extract_text_doc("Contoh-Surat-Penawaran-PDF.pdf")
print(pdf_text_ext[:1000])


PT. KARYA ABADI
Perusahaan Properti dan Real Estate
Jl. Danau Seberang No. 123, Bogor
Telp. (0251) 232323 | Fax. (0251) 898989
Email: marketing@karyaabadi.co.id
NPWP: 01.422.1.344.111
Bogor, 12 Januari 2018
Nomor : PKA/021/AB-12Y/PP/I/2018
Lampiran : 3 (tiga) rangkap
Perihal : Penawaran barang
Kepada Yth.
Direktur Utama
PT. Dinamika Surya
Di tempat.
Dengan Hormat,
Bersama surat ini, kami PT. Karya Abadi sebagai perusahaan yang bergerak di bidang
properti dan real estate mengucapkan terima kasih atas kesempatan yang Anda
berikan untuk kami dalam memperkenalkan produk unggulan perusahaan.
PT. Karya Abadi merupakan perusahaan penyedia properti dan real estate yang sudah
berkembang sejak tahun 2010. Kami telah banyak bekerja sama dengan berbagai
kalangan masyarakat dalam memberikan produk berupa tanah dan bangunan yang
berkualitas tinggi dan menjanjikan. Kami yakin bahwa produk kami juga dapat
memenuhi dan menunjang kebutuhan PT. Dinamika Surya.
Sehubungan dengan hal itu, kami menawarkan s

## Split teks menjadi chunk

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
split_text = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
dokument = split_text.create_documents([pdf_text_ext])
print(f"total potongan : {len(dokument)}")


total potongan : 5


In [46]:
dokument

[Document(metadata={}, page_content='PT. KARYA ABADI\nPerusahaan Properti dan Real Estate\nJl. Danau Seberang No. 123, Bogor\nTelp. (0251) 232323 | Fax. (0251) 898989\nEmail: marketing@karyaabadi.co.id\nNPWP: 01.422.1.344.111\nBogor, 12 Januari 2018\nNomor : PKA/021/AB-12Y/PP/I/2018\nLampiran : 3 (tiga) rangkap\nPerihal : Penawaran barang\nKepada Yth.\nDirektur Utama\nPT. Dinamika Surya\nDi tempat.\nDengan Hormat,\nBersama surat ini, kami PT. Karya Abadi sebagai perusahaan yang bergerak di bidang'),
 Document(metadata={}, page_content='properti dan real estate mengucapkan terima kasih atas kesempatan yang Anda\nberikan untuk kami dalam memperkenalkan produk unggulan perusahaan.\nPT. Karya Abadi merupakan perusahaan penyedia properti dan real estate yang sudah\nberkembang sejak tahun 2010. Kami telah banyak bekerja sama dengan berbagai\nkalangan masyarakat dalam memberikan produk berupa tanah dan bangunan yang\nberkualitas tinggi dan menjanjikan. Kami yakin bahwa produk kami juga dapat'

## membuat vector embedding dan faiss index

In [47]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embed_text = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_text = FAISS.from_documents(dokument, embed_text)


In [30]:
embed_text 

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
vector_text

## menggunakan model ollama

In [31]:
from langchain_community.chat_models import ChatOllama
llm_model= ChatOllama(
    base_url="http://localhost:11434",
    model="llama3",
    temperature=0.1
)


## membuat pipeline Question Answering (QA)

In [36]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_model,
    retriever=vector_text.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)


1. as_retriever: mengubah vector menjadi retriever
2. search_kwargs={"k": 3}: menampilkan top 3 dokument yang paling cocok dengan pertanyaan user


In [37]:
pertanyaan = "Siapa pengirim surat ini dan apa isi utamanya?"
hasil = qa_chain.invoke({"query": pertanyaan})

print("Jawaban:\n")
print(hasil['result'])


Jawaban:

Pengirim surat ini adalah PT. Karya Abadi, sebuah perusahaan penyedia properti dan real estate. Isi utama surat ini adalah penawaran produk unggulan mereka, yaitu Golden Hills Residence di Bogor, yang terdiri dari berbagai unit dengan luas bangunan dan tanah yang berbeda-beda.


In [41]:
pertanyaan = "Apa tujuan surat ini?"
hasil = qa_chain.invoke({"query": pertanyaan})

print("Jawaban:\n")
print(hasil['result'])

Jawaban:

Tujuan surat ini adalah untuk menawarkan produk unggulan PT. Karya Abadi, yaitu Golden Hills Residence — Bogor, kepada Direktur Utama PT. Dinamika Surya dan memenuhi kebutuhan perusahaan tersebut.


In [48]:
pertanyaan = "penawarannya apa dan kriteria produknya apa"
hasil = qa_chain.invoke({"query": pertanyaan})

print("Jawaban:\n")
print(hasil['result'])

Jawaban:

Based on the context, it appears that PT. Karya Abadi is offering a property product called "Golden Hills Residence — Bogor". The criteria of this product are:

* Types of buildings: 100 m2, 200 m2, 300 m2, and 400 m2
* Land sizes: 200 m2, 300 m2, 400 m2, and 500 m2
* Certificate: SHM (Sertifikat Hak Milik)
* Building specifications: attached document
* Total units: 10 units
* Availability of units: 8 units
* Location: Jl. Raya Timur No. 10, Bogor

The product seems to be a residential property development in Bogor, with various sizes and amenities.
